In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
np.random.seed(123)
tf.random.set_seed(123)

In [ ]:
def baseline_model():
    model = Sequential()
    # Convolutional layers
    model.add(layers.Input(shape=(128, 299, 299, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Flatten and Fully Connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout layer to prevent overfitting
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
model = build_baseline_model()
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=30)

In [ ]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
# Plot training & validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
##Hyperparameter tuning

In [ ]:
# Set EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
def tune_model(dropout_rate=0.5, learning_rate=0.001, batch_size=32):
    # Rebuild the model with the specified dropout rate
    model = build_baseline_model()
    
    # Compile the model with the specified learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    # Train the model with the specified batch size and EarlyStopping
    history = model.fit(train_generator, 
                        validation_data=val_generator, 
                        epochs=30, 
                        batch_size=batch_size,
                        callbacks=[early_stopping])
    
    return history

In [ ]:
# Save the model
model.save('baseline_model.h5')

In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model('baseline_model.h5')

In [ ]:
# Evaluate on test dataset
test_loss, test_accuracy = loaded_model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# Make predictions on the test set
y_pred = loaded_model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels

In [ ]:
# True labels
y_true = test_generator.classes

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'COVID'], yticklabels=['Normal', 'COVID'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()